In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso

from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [18]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [12]:
#df.describe
df.shape

(1154112, 8)

In [6]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60 )

In [11]:
Aver = df['duration'].mean()
print(Aver)

19.1672240937939


In [25]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

In [42]:
#df.isna().mean().round(4) * 100
df['PUlocationID'].value_counts()

-1.0      958267
 221.0      8551
 206.0      7128
 129.0      5455
 92.0       5224
           ...  
 111.0         6
 34.0          5
 27.0          4
 2.0           2
 110.0         1
Name: PUlocationID, Length: 262, dtype: int64

In [43]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60 )

    df = df[(df.duration > 1) & (df.duration <60)]

    categorical = ['PUlocationID','DOlocationID']


    df[categorical] = df[categorical].astype(str)
    
    return df

In [44]:
df_train = read_dataframe('./data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/fhv_tripdata_2021-02.parquet')

In [48]:
categorical = ['PUlocationID','DOlocationID']
    
dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient = 'records')
x_val = dv.transform(val_dicts)

In [52]:
x_train.shape, x_val.shape

((1106890, 525), (987770, 525))

In [49]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [50]:
lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_val)

mean_squared_error(y_val, y_pred, squared = False)

10.897037683801184

In [53]:
lr = LinearRegression()
lr.fit(x_val, y_val)

y_pred = lr.predict(x_val)

mean_squared_error(y_val, y_pred, squared = False)

10.84046925848122